In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from data_generators.standard_generator import StandardDataGenerator
from environments.contextual_environment import ContextualEnvironment
from learners.pricing.contextual_learner import ContextualLearner
from learners.pricing.thompson_sampling import ThompsonSampling
from learners.pricing.ucb import UCB
from utils.context_generator_v2 import ContextGenerator 

In [2]:
src = 'src/basic005.json'

In [3]:
dg = StandardDataGenerator(filename=src)

In [4]:
selected_bid = 3
T = 150
n_experiments = 1
learners_to_test = [ThompsonSampling]
cg_start_from = 50
cg_confidence = 0.01
cg_frequency = 10

features = dg.get_features() 
fractions = dg.get_class_distributions(selected_bid)
bandit_args = {
            'arm_values': dg.get_margins()
}
env_args = {
    'bid_idx': selected_bid,
    'mode': 'all',
    'src': dg.get_source(),
    'generator': 'standard'
}

In [5]:
rewards_per_experiment = {}
context_split_per_experiment = {}
days_of_split = {}
for learner in learners_to_test:
    rewards_per_experiment[learner.LEARNER_NAME] = []
    context_split_per_experiment[learner.LEARNER_NAME] = []
    days_of_split[learner.LEARNER_NAME] = []

In [6]:
leaves_feature = {}
leaves_feature[True,True] = "C1"
leaves_feature[True,False] = "C3"
leaves_feature[False,True] = "C3"
leaves_feature[False,False] = "C2"
print(leaves_feature)



int_classes = {}
int_classes["C1"] = 0
int_classes["C2"] = 1
int_classes["C3"] = 2

int_to_feature_1 = {}
int_to_feature_1[0] = True
int_to_feature_1[1] = True
int_to_feature_1[2] = False
int_to_feature_1[3] = False

int_to_feature_2 = {}
int_to_feature_2[0] = True
int_to_feature_2[1] = False
int_to_feature_2[2] = True
int_to_feature_2[3] = False

splitted_rewards_per_experiment = {}
splitted_rewards_per_experiment[True,True] = []
splitted_rewards_per_experiment[True,False] = []
splitted_rewards_per_experiment[False,True] = []
splitted_rewards_per_experiment[False,False] = []

opt_per_experiment = {}
opt_per_experiment[True,True] = []
opt_per_experiment[True,False] = []
opt_per_experiment[False,True] = []
opt_per_experiment[False,False] = []

{(True, True): 'C1', (True, False): 'C3', (False, True): 'C3', (False, False): 'C2'}


In [7]:
margins = dg.get_margins()
classes = dg.get_classes()
conversion_rates = dg.get_conversion_rates(mode='all')
future_purchases = dg.get_future_purchases(mode='all')
selected_bid = 3
number_of_clicks = dg.get_daily_clicks(mode='all')
costs_per_click = dg.get_costs_per_click(mode='aggregate', bid=selected_bid)
fixed_cost = costs_per_click[selected_bid]
fixed_n_clicks = np.rint(dg.get_daily_clicks(mode='aggregate')[selected_bid]).astype(int)
fractions = dg.get_class_distributions(selected_bid)

temp = (margins * np.average(conversion_rates * (1 + future_purchases),
                             axis=0,
                             weights=fractions) - fixed_cost) * fixed_n_clicks
aggr_opt_arm = np.argmax(temp)
aggr_opt = np.max(temp)

disaggr_opt = 0
opt_arms = []
for i, _ in enumerate(conversion_rates):
    t = (margins * conversion_rates[i] * (1 + future_purchases[i]) - fixed_cost) * \
                number_of_clicks[i, selected_bid]
    opt_arm = np.argmax(t)
    opt_value = np.max(t)
    opt_arms.append(opt_arm)
    disaggr_opt += opt_value

In [8]:
for e in range(n_experiments):
    # Initialization of the learners to test and their related environment:
    # the list is composed of tuples (Learner, Environment)
    print(f'\n\nrunning experiment {e + 1}/{n_experiments}...')
    test_instances = []
    
    for learner in learners_to_test:
        context_learner = ContextualLearner(features, learner, **bandit_args)
        test_instances.append(
            (context_learner,
             ContextualEnvironment(**env_args),
             ContextGenerator(features=features,
                              contextual_learner=context_learner,
                              update_frequency=cg_frequency,
                              start_from=cg_start_from,
                              confidence=cg_confidence,
                              verbose=0))
        )
    for t in range(T):
        for context_learner, env, context_generator in test_instances:
            print(f'##### DAY {t} ######')
            context_learner.next_day()
            past_arms = None
            past_features = None
            month_purchases = env.get_next_purchases_at_day(t, keep=False)
            if month_purchases is not None:
                past_arms = env.get_selected_arms_at_day(t - 30, keep=False)
                past_features = env.get_collected_user_features_at_day(t - 30, keep=False)
                context_learner.update_next_purchases(past_arms, month_purchases, past_features)

            pulled_arms = context_learner.pull_arms()
            daily_rewards = env.day_round(pulled_arms)
            daily_users_features = env.get_collected_user_features_at_day(t)
            daily_pulled_arms = env.get_selected_arms_at_day(t)

            context_learner.update(daily_rewards, daily_pulled_arms, daily_users_features)
            context_generator.collect_daily_data(daily_pulled_arms, daily_rewards, daily_users_features,
                                                         next_purchases=month_purchases, past_pulled_arms=past_arms,
                                                         past_features=past_features)
    for learner, _, _ in test_instances:
        learner.next_day()
        rewards_per_experiment[learner.base_learner_class.LEARNER_NAME].append(learner.get_daily_rewards())
        context_split_per_experiment[learner.base_learner_class.LEARNER_NAME].append(learner.get_splits_count())
        days_of_split[learner.base_learner_class.LEARNER_NAME].append(context_generator.metadata['SPLIT_DAYS'])
    # end -> save rhe results.
        if learner.base_learner_class.LEARNER_NAME == 'TS-Pricing':
        #split_day=learner.context_tree.collected_splits[0]+16#should be a list, returns only one value??
            for leaf in learner.context_tree.get_leaves():
                print(leaf.feature_subspace)
                feature_2=leaf.feature_subspace['feature-2']
                feature_1=leaf.feature_subspace['feature-1']
                leaf_class=leaves_feature[feature_1,feature_2]
                leaf_values=leaf.get_daily_rewards()
                print(len(leaf_values))
                split_day=365-len(leaf_values)
                print(split_day)
                splitted_rewards_per_experiment[feature_1,feature_2].append(leaf_values) #save rewards for specific leaf
                opt_single = opt_arms[int_classes[leaf_class]] #calculate opt for specific leaf 
                opt=[]
                for relative_day,val in enumerate(leaf_values): #
                    curr_day=split_day+relative_day
                    custom_clicks=env.collected_users_categories[curr_day,leaf_class] #call function on the env that returns n of clicks for current day
                    optimal_value= opt_single * custom_clicks
                    opt.append(optimal_value)
                opt_per_experiment[feature_1,feature_2].append(opt) #save list of optimal values for the current leaf specified by her features





running experiment 1/1...
##### DAY 0 ######
##### DAY 1 ######
##### DAY 2 ######
##### DAY 3 ######
##### DAY 4 ######
##### DAY 5 ######
##### DAY 6 ######
##### DAY 7 ######
##### DAY 8 ######
##### DAY 9 ######
##### DAY 10 ######
##### DAY 11 ######
##### DAY 12 ######
##### DAY 13 ######
##### DAY 14 ######
##### DAY 15 ######
##### DAY 16 ######
##### DAY 17 ######
##### DAY 18 ######
##### DAY 19 ######
##### DAY 20 ######
##### DAY 21 ######
##### DAY 22 ######
##### DAY 23 ######
##### DAY 24 ######
##### DAY 25 ######
##### DAY 26 ######
##### DAY 27 ######
##### DAY 28 ######
##### DAY 29 ######
##### DAY 30 ######
##### DAY 31 ######
##### DAY 32 ######
##### DAY 33 ######
##### DAY 34 ######
##### DAY 35 ######
##### DAY 36 ######
##### DAY 37 ######
##### DAY 38 ######
##### DAY 39 ######
##### DAY 40 ######
##### DAY 41 ######
##### DAY 42 ######
##### DAY 43 ######
##### DAY 44 ######
##### DAY 45 ######
##### DAY 46 ######
##### DAY 47 ######
##### DAY 48 ######
##

#420: [(True, False)] -> {'feature-1': True}
#421: [(True, False)] -> {'feature-1': True}
#422: [(True, True)] -> {'feature-1': True}
#423: [(True, True)] -> {'feature-1': True}
#424: [(True, True)] -> {'feature-1': True}
#425: [(False, False)] -> {'feature-1': False}
#426: [(False, False)] -> {'feature-1': False}
#427: [(True, True)] -> {'feature-1': True}
#428: [(False, False)] -> {'feature-1': False}
#429: [(False, False)] -> {'feature-1': False}
#430: [(False, False)] -> {'feature-1': False}
#431: [(True, False)] -> {'feature-1': True}
#432: [(False, True)] -> {'feature-1': False}
#433: [(True, True)] -> {'feature-1': True}
#434: [(False, True)] -> {'feature-1': False}
#435: [(True, True)] -> {'feature-1': True}
#436: [(True, True)] -> {'feature-1': True}
#437: [(True, True)] -> {'feature-1': True}
#438: [(False, False)] -> {'feature-1': False}
#439: [(True, False)] -> {'feature-1': True}
#440: [(True, True)] -> {'feature-1': True}
#441: [(True, False)] -> {'feature-1': True}
#442:

#40: [(True, True)] -> {'feature-1': True}
#41: [(False, True)] -> {'feature-1': False}
#42: [(False, False)] -> {'feature-1': False}
#43: [(False, False)] -> {'feature-1': False}
#44: [(False, False)] -> {'feature-1': False}
#45: [(True, True)] -> {'feature-1': True}
#46: [(False, False)] -> {'feature-1': False}
#47: [(True, False)] -> {'feature-1': True}
#48: [(True, True)] -> {'feature-1': True}
#49: [(False, True)] -> {'feature-1': False}
#50: [(True, True)] -> {'feature-1': True}
#51: [(True, True)] -> {'feature-1': True}
#52: [(True, False)] -> {'feature-1': True}
#53: [(False, True)] -> {'feature-1': False}
#54: [(True, True)] -> {'feature-1': True}
#55: [(False, False)] -> {'feature-1': False}
#56: [(False, True)] -> {'feature-1': False}
#57: [(False, False)] -> {'feature-1': False}
#58: [(True, True)] -> {'feature-1': True}
#59: [(True, False)] -> {'feature-1': True}
#60: [(False, False)] -> {'feature-1': False}
#61: [(False, True)] -> {'feature-1': False}
#62: [(True, True)] 

#69: [(True, True)] -> {'feature-1': True}
#70: [(False, False)] -> {'feature-1': False}
#71: [(False, False)] -> {'feature-1': False}
#72: [(True, False)] -> {'feature-1': True}
#73: [(True, True)] -> {'feature-1': True}
#74: [(False, False)] -> {'feature-1': False}
#75: [(True, True)] -> {'feature-1': True}
#76: [(True, True)] -> {'feature-1': True}
#77: [(False, False)] -> {'feature-1': False}
#78: [(True, True)] -> {'feature-1': True}
#79: [(True, True)] -> {'feature-1': True}
#80: [(False, False)] -> {'feature-1': False}
#81: [(False, True)] -> {'feature-1': False}
#82: [(False, False)] -> {'feature-1': False}
#83: [(True, True)] -> {'feature-1': True}
#84: [(True, True)] -> {'feature-1': True}
#85: [(True, True)] -> {'feature-1': True}
#86: [(True, True)] -> {'feature-1': True}
#87: [(True, False)] -> {'feature-1': True}
#88: [(False, False)] -> {'feature-1': False}
#89: [(True, False)] -> {'feature-1': True}
#90: [(True, True)] -> {'feature-1': True}
#91: [(False, False)] -> {'f

#274: [(False, False)] -> {'feature-1': False}
#275: [(True, True)] -> {'feature-1': True}
#276: [(False, True)] -> {'feature-1': False}
#277: [(False, False)] -> {'feature-1': False}
#278: [(False, False)] -> {'feature-1': False}
#279: [(False, False)] -> {'feature-1': False}
#280: [(True, True)] -> {'feature-1': True}
#281: [(True, False)] -> {'feature-1': True}
#282: [(True, True)] -> {'feature-1': True}
#283: [(True, True)] -> {'feature-1': True}
#284: [(True, True)] -> {'feature-1': True}
#285: [(False, True)] -> {'feature-1': False}
#286: [(False, False)] -> {'feature-1': False}
#287: [(True, True)] -> {'feature-1': True}
#288: [(False, False)] -> {'feature-1': False}
#289: [(False, True)] -> {'feature-1': False}
#290: [(True, True)] -> {'feature-1': True}
#291: [(True, False)] -> {'feature-1': True}
#292: [(True, False)] -> {'feature-1': True}
#293: [(True, False)] -> {'feature-1': True}
#294: [(False, False)] -> {'feature-1': False}
#295: [(False, False)] -> {'feature-1': False

...

#152: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#153: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#154: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#155: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#156: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#157: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#158: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#159: [(True, False)] -> {'feature-1': True, 'feature-2': False}
#160: [(True, False)] -> {'feature-1': True, 'feature-2': False}
#161: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#162: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#163: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#164: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#165: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#166: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#167: [(False, True)] -> {'feature-

#0: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#1: [(True, False)] -> {'feature-1': True, 'feature-2': False}
#2: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#3: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#4: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#5: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#6: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#7: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#8: [(True, False)] -> {'feature-1': True, 'feature-2': False}
#9: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#10: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#11: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#12: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#13: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#14: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#15: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#16:

#315: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#316: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#317: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#318: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#319: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#320: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#321: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#322: [(True, False)] -> {'feature-1': True, 'feature-2': False}
#323: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#324: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#325: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#326: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#327: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#328: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#329: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#330: [(False, False)] -> {

#433: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#434: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#435: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#436: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#437: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#438: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#439: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#440: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#441: [(False, True)] -> {'feature-1': False, 'feature-2': True}
#442: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#443: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#444: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#445: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#446: [(False, False)] -> {'feature-1': False, 'feature-2': False}
#447: [(True, True)] -> {'feature-1': True, 'feature-2': True}
#448: [(True, True)] -> {'featu

KeyError: (316, 'C2')

In [ ]:

env.collected_users_categories[320,'C2']

In [ ]:
for e in range(n_experiments):
    for i in range(4):
        feature_1=int_to_feature_1[i]
        feature_2=int_to_feature_2[i]
        for j in range(365-len(splitted_rewards_per_experiment[feature_1,feature_2][e])):
            splitted_rewards_per_experiment[feature_1,feature_2][e]=np.insert(splitted_rewards_per_experiment[feature_1,feature_2][e],0,0,axis=0)
            opt_per_experiment[feature_1,feature_2][e]=np.insert(opt_per_experiment[feature_1,feature_2][e],0,0,axis=0)
          
                   

In [ ]:


for i in range(4):
    feature_1 = int_to_feature_1[i]
    feature_2 = int_to_feature_2[i]
    opt = splitted_rewards_per_experiment[feature_1,feature_2]
    val = opt_per_experiment[feature_1,feature_2]
    plt.figure(figsize=(10,8))
    plt.ylabel("Regret")
    plt.xlabel("day")

    plt.plot(np.cumsum(np.mean(np.array(opt) - np.array(val), axis=0)))

    plt.legend(rewards_per_experiment.keys())
    plt.show()

In [ ]:
plt.figure()
plt.ylabel("num_splits")
plt.xlabel("experiment")
for val in context_split_per_experiment.values():
    plt.plot(val, '--o')
plt.legend(context_split_per_experiment.keys())
plt.show()

In [ ]:
for key in context_split_per_experiment:
    print(f'{key}: avg splits -> {np.mean(context_split_per_experiment[key])}')

In [ ]:
val = days_of_split[ThompsonSampling.LEARNER_NAME]
avg_days = np.mean(val, axis=0)
val, avg_days

In [ ]:
# Plotting the results
plt.figure(figsize=(10,8))
plt.ylabel("Regret")
plt.xlabel("day")
for val in rewards_per_experiment.values():
    plt.plot(np.cumsum(np.mean(disaggr_opt - val, axis=0)))

#plt.axvline(x=cg_start_from, color='r', linestyle='--')
for val in days_of_split.values():
    avg_days = np.mean(val, axis=0)
    for day in avg_days:
        plt.axvline(x=day, color='m', linestyle='--')

plt.legend(rewards_per_experiment.keys())
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.xlabel("day")
plt.ylabel("daily reward")
plt.plot([disaggr_opt]*T, '--g', label='clairwoyant')
plt.plot([aggr_opt]*T, '--c', label='aggr_clairwoyant')
for key in rewards_per_experiment:
    plt.plot(np.mean(rewards_per_experiment[key], axis=0), label=key)

#plt.axvline(x=cg_start_from, color='r', linestyle='--')
for val in days_of_split.values():
    avg_days = np.mean(val, axis=0)
    for day in avg_days:
        plt.axvline(x=day, color='m', linestyle='--')

plt.legend(loc='best')
plt.show()